## Налаштування та завантаження даних

In [1]:
import pandas as pd
import numpy as np
import os
import ipynbname

In [2]:
print(os.listdir())

['.ipynb_checkpoints', '2025 Developer Survey Tool.pdf', 'homework_1.ipynb', 'Lesson_file.ipynb', 'Practice_1.ipynb', 'stack-overflow-developer-survey-2025.zip', 'survey_results_public.csv', 'survey_results_schema.csv']


In [3]:
# Завантажуємо дані
try:
    df = pd.read_csv('survey_results_public.csv')
    schema = pd.read_csv('survey_results_schema.csv')
except FileNotFoundError:
    print(f'Помилка: Файли "survey_results_public.csv" або "survey_result_schema.csv" не знайдено')

In [4]:
# Встановлюємо ResponseId як індекс для зручності
df = df.set_index('ResponseId')

# Встановлюємо qname як індекс для схеми
schema = schema.set_index('qname')

In [5]:
# Переглянемо дані
print('Основні дані (df): ')
print(df.head())

print('\nСхема (schema): ')
print(schema.head())

Основні дані (df): 
                                MainBranch              Age  \
ResponseId                                                    
1           I am a developer by profession  25-34 years old   
2           I am a developer by profession  25-34 years old   
3           I am a developer by profession  35-44 years old   
4           I am a developer by profession  35-44 years old   
5           I am a developer by profession  35-44 years old   

                                                    EdLevel  \
ResponseId                                                    
1           Master’s degree (M.A., M.S., M.Eng., MBA, etc.)   
2                       Associate degree (A.A., A.S., etc.)   
3              Bachelor’s degree (B.A., B.S., B.Eng., etc.)   
4              Bachelor’s degree (B.A., B.S., B.Eng., etc.)   
5           Master’s degree (M.A., M.S., M.Eng., MBA, etc.)   

                                                   Employment  \
ResponseId                     

In [6]:
# Перевіримо типи даних та пропуски
print('\nІнформація про df: ')
df.info()


Інформація про df: 
<class 'pandas.core.frame.DataFrame'>
Index: 49123 entries, 1 to 49123
Columns: 169 entries, MainBranch to JobSat
dtypes: float64(50), object(119)
memory usage: 63.7+ MB


## Відповіді на основні запитання

In [8]:
# 1. Скільки респондентів пройшли опитування?
total_respondents = len(df)

print(f'1. Загальна кількість респондентів: {total_respondents}')

1. Загальна кількість респондентів: 49123


In [9]:
# 2. Скільки респондентів відповіли на всі запитання?
schema_questions = set(schema.index.dropna())
df_columns = set(df.columns)
relevant_columns = list(schema_questions.intersection(df_columns))
respondents_all_questions = df[relevant_columns].dropna()
count_all_questions = len(respondents_all_questions)

print(f'2. Кількість респондентів, що відповіли на всі запитання зі схеми: {count_all_questions}')

2. Кількість респондентів, що відповіли на всі запитання зі схеми: 0


In [12]:
# 3. Які значення мір центральної тенденції для досвіду (WorkExp)?
work_exp_numeric = pd.to_numeric(df['WorkExp'], errors='coerce')

mean_exp = work_exp_numeric.mean()
median_exp = work_exp_numeric.median()
mode_exp = work_exp_numeric.mode().tolist()

print('3. Міри центральної тенденції для досвіду (WorkExp):')
print(f'  - Середнє (Mean): {mean_exp:.2f} років')
print(f'  - Медіана (Median): {median_exp} років')
print(f'  - Мода (Mode): {mode_exp} років')

3. Міри центральної тенденції для досвіду (WorkExp):
  - Середнє (Mean): 13.37 років
  - Медіана (Median): 10.0 років
  - Мода (Mode): [10.0] років


In [17]:
# 4. Скільки респондентів працює віддалено?
print('4. Аналіз стовпця "RemoteWork":')
print(df['RemoteWork'].value_counts())
remote_count = df[df['RemoteWork'] == 'Remote'].shape[0]
print(f"\nКількість респондентів, що працюють повністю віддалено ('Remote'): {remote_count}")

4. Аналіз стовпця "RemoteWork":
RemoteWork
Remote                                                                          10924
Hybrid (some remote, leans heavy to in-person)                                   6728
In-person                                                                        6034
Hybrid (some in-person, leans heavy to flexibility)                              5824
Your choice (very flexible, you can come in when you want or just as needed)     4237
Name: count, dtype: int64

Кількість респондентів, що працюють повністю віддалено ('Remote'): 10924


In [18]:
# 5. Який відсоток респондентів програмує на Python?
lang_col = 'LanguageHaveWorkedWith'
valid_lang_responses = df[lang_col].dropna()
python_users = valid_lang_responses.str.contains('Python', na=False)

python_count = python_users.sum()
total_valid_respondents = len(valid_lang_responses)

python_percentage = (python_count / total_valid_respondents) * 100

print(f'5. Відсоток респондентів, що програмують на Python: {python_percentage:.2f}%')
print(f'(Це {python_count} із {total_valid_respondents} респондентів, що відповіли на питання про мови)')

5. Відсоток респондентів, що програмують на Python: 58.29%
(Це 18444 із 31643 респондентів, що відповіли на питання про мови)


In [19]:
# 6. Скільки респондентів навчалося програмувати за допомогою онлайн курсів?
learn_col = 'LearnCode'
online_learners = df[learn_col].str.contains('Online Courses', na=False)
online_learners_count = online_learners.sum()

print(f'6. Кількість респондентів, що навчалися за допомогою онлайн-курсів: {online_learners_count}')

6. Кількість респондентів, що навчалися за допомогою онлайн-курсів: 10957


In [20]:
# 7. Середня та медіанна компенсація (ConvertedCompYearly) Python-розробників по країнам?
comp_col = 'ConvertedCompYearly'
if not pd.api.types.is_numeric_dtype(df[comp_col]):
    df[comp_col] = pd.to_numeric(df[comp_col], errors='coerce')

lang_col = 'LanguageHaveWorkedWith'
python_users_mask = df[lang_col].str.contains('Python', na=False)

python_df = df[python_users_mask]

country_comp = python_df.groupby('Country')[comp_col].agg(['mean', 'median'])
country_comp_sorted = country_comp.sort_values(by='median', ascending=False)
print('7. Середня та медіанна компенсація Python-розробників по країнам (Top 20):')
print(country_comp_sorted.head(20))

7. Середня та медіанна компенсація Python-розробників по країнам (Top 20):
                                                             mean    median
Country                                                                    
Oman                                                390135.000000  390135.0
Andorra                                             226103.500000  226103.5
United States of America                            173295.260243  150000.0
Israel                                              135828.365385  142594.0
Switzerland                                         156456.600000  142592.0
Nomadic                                             120131.571429  139218.0
Ireland                                             120523.918919  116015.0
Luxembourg                                          116014.714286  109054.0
Kyrgyzstan                                          106008.500000  106008.5
Belize                                              102121.000000  102121.0
Belarus      

In [21]:
# 8. Які рівні освіти мають 5 респондентів з найбільшою компенсацією?
comp_col = 'ConvertedCompYearly'
edu_col = 'EdLevel'

top_5_comp = df.sort_values(by=comp_col, ascending=False).head(5)

print('8. Рівні освіти 5 респондентів з найбільшою компенсацією:')
print(top_5_comp[[edu_col, comp_col]])

8. Рівні освіти 5 респондентів з найбільшою компенсацією:
                                                    EdLevel  \
ResponseId                                                    
34268                   Associate degree (A.A., A.S., etc.)   
28701       Master’s degree (M.A., M.S., M.Eng., MBA, etc.)   
43144                   Associate degree (A.A., A.S., etc.)   
35354          Bachelor’s degree (B.A., B.S., B.Eng., etc.)   
45972       Master’s degree (M.A., M.S., M.Eng., MBA, etc.)   

            ConvertedCompYearly  
ResponseId                       
34268                50000000.0  
28701                33552715.0  
43144                18387548.0  
35354                15430267.0  
45972                13921760.0  


## Бонусні запитання

In [22]:
# 1. В кожній віковій категорії, який відсоток респондентів програмує на Python?
age_col = 'Age'
lang_col = 'LanguageHaveWorkedWith'

df['UsesPython'] = df[lang_col].str.contains('Python', na=False)

age_python_percentage = df.groupby(age_col)['UsesPython'].mean() * 100
age_python_percentage = age_python_percentage.sort_index()

print('Бонус 1: Відсоток респондентів, що програмують на Python, по віковим категоріям:')
print(age_python_percentage)

Бонус 1: Відсоток респондентів, що програмують на Python, по віковим категоріям:
Age
18-24 years old      40.032626
25-34 years old      36.942675
35-44 years old      36.706469
45-54 years old      38.659218
55-64 years old      37.242955
65 years or older    31.634820
Prefer not to say    31.216931
Name: UsesPython, dtype: float64


In [23]:
# 2. Серед респондентів у 75+ перцентилі компенсації, що працюють віддалено, які індустрії найпоширеніші?
comp_col = 'ConvertedCompYearly'
remote_col = 'RemoteWork'
industry_col = 'Industry'

comp_75th_percentile = df[comp_col].quantile(0.75)
print(f'Бонус 2: 75-й перцентиль компенсації: {comp_75th_percentile:,.2f}')

high_comp_df = df[df[comp_col] >= comp_75th_percentile]
high_comp_remote_df = high_comp_df[high_comp_df[remote_col] == 'Remote']
common_industries = high_comp_remote_df[industry_col].value_counts()

print('Найпоширеніші індустрії для "Remote" з топ 25% компенсації:')
print(common_industries.head(10))

Бонус 2: 75-й перцентиль компенсації: 120,630.25
Найпоширеніші індустрії для "Remote" з топ 25% компенсації:
Industry
Software Development                          1184
Fintech                                        189
Healthcare                                     188
Other:                                         176
Internet, Telecomm or Information Services     138
Banking/Financial Services                      88
Government                                      78
Media & Advertising Services                    75
Retail and Consumer Services                    64
Transportation, or Supply Chain                 63
Name: count, dtype: int64
